In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
# note scale = 2, or 3 makes it more precise, but too big.
scales = [1]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [6]:
# Verifier/ data consumer side:
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    # can AND condition! (same for OR as well)
    filter = torch.logical_and((x>50), (y>300))
    filtered_x = s.where(filter, x)
    filtered_y = s.where(filter, y)
    return s.covariance(filtered_x, filtered_y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)


verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [7]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 1, param_scale: 1, scale_input_multiplier: 10) ------------->

+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error  | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.08456421 | 0.16912842   | 0.16912842 | 0         | 0.08456421     | 0.16912842       | 0.16912842    | 0             | 0.014302211        | 0.00025573664      | 0.00025573664          |
+------------+--------------+------------+-----------+----------------+------------------+---------------+-------------

==== Generate & Calibrate Setting ====
scale:  [1]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":1,"param_scale":1,"scale_rebase_multiplier":10,"lookup_range":[-1348,778],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":13120,"total_assignments":3317,"total_const_size":608,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,1],"model_input_scales":[1,1],"module_sizes":{"kzg":[],"poseidon":[13120,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709724643678}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.4659230709075928 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 330.5
==== Generating Proof ====
proof:  {'instances': [['b8c4693b3424042141e265e5179743499536c8b9f1ddc7d4d4fc847c1c09df04', '7d055c183092753c601560dfbfa0b446f5417438227d99435c476a0659de3d14', '0100000000000000000000000000000000000000000000000000000000000000', '9502000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1f2cdb3c1a98ca9febc6508ec1fb81f9b428e1f123b7dff72e3597d7195beb0000014197b88d8f5fc5dfd8aeb29dab05169ab1bc9214e763bce3c0291684efa92468ff4c9ace3b9e1ada390ef08f83fc7bcc7026c5541bc45ee5cf9f3a2bb23b0b9a064b997d78f080c8f6c076fd824c2095c89f418d51045874a6192aed72d4219f6fe9b1fbc52f4b93660b42444220116f8fc4a03dde7433ddd43c2f33d5cf28e13534befdba8d0d964a11cb9f2ba48ec684552c46f92680c98631c5098f042f2b76ad2ca25e362f67e755fbdc5444528757dc86a58fe5b2bdca5ab35610b80f72cd7f20fd25a2817564fb930384a5d2771bc22370d5d16f67c1349a865a7e2a6816cd0d

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [330.5]
